In [1]:
# Import the required libraries
from google.colab import drive
import pandas as pd
import numpy as np
import csv

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
file_path = '/content/drive/MyDrive/557 Project - Group 44/Top100stocks_indicator.csv' #NIFTYBEES.csv' #Top100stocks_indicator.csv'
df_all = pd.read_csv(file_path)

In [4]:
# df.head
unique_names = df_all['SYMBOL'].unique()

In [5]:
# df.head

In [6]:
# unique_names

In [7]:
# df = pd.read_csv('RSI_CCI_MACD.csv')

In [8]:
# df.tail()

In [9]:
def indicator_analysis(top_25_stocks, stock, df, Start_idx, rows, cci_threshold, rsi_threshold, percent_profit, percent_SL):
  print(f'Performing For {stock}')

  idx = Start_idx

  while idx < Start_idx+len(df):
    #print(idx)
    if df.loc[idx, 'CCI'] >= cci_threshold and df.loc[idx, 'RSI'] >= rsi_threshold:
      hold_idx = idx
      count_tp_breach = 0
      count_sl_breach = 0
      date = None
      buy_price = None
      target_price = None
      stop_loss_price = None
      exited_value = None
      stock_price_indicator_less_threshold = None
      profit_or_loss = None
      exited_value_date = None

      date = df.loc[idx, 'TIMESTAMP'] # Got the date for that particular index
      buy_price = df.loc[idx, 'CLOSE'] # Got buy price for that particular index
      target_price = buy_price + (percent_profit/100) * buy_price # Calculated target price for the buy price
      stop_loss_price = buy_price - (percent_SL/100) * buy_price # Calculated stelp loss price for the buy price

      while idx < Start_idx+len(df) and df.loc[idx, "CLOSE"] < target_price and df.loc[idx, "CLOSE"] > stop_loss_price:
        idx += 1
      
      if idx == Start_idx+len(df):
        break         # What to do?
      
      elif df.loc[idx, "CLOSE"] >= target_price:
        exited_value = df.loc[idx, "CLOSE"]
        exited_value_date = df.loc[idx, "TIMESTAMP"]
        count_tp_breach += 1
      
      elif df.loc[idx, "CLOSE"] <= stop_loss_price:
        exited_value = df.loc[idx, "CLOSE"]
        exited_value_date = df.loc[idx, "TIMESTAMP"]
        count_sl_breach += 1

      idx1 = hold_idx+1
      # CCI_max = df.loc[hold_idx, indicator]
      # CCI_max_value = df.loc[hold_idx, "CLOSE"]
      while idx1 < Start_idx+len(df):
        #print("F1", idx1, Start_idx+len(df))
        if np.isnan(df.loc[idx1, 'CCI']) == True or np.isnan(df.loc[idx1, 'RSI']) == True:
          # print("F1", df.loc[idx1, "CLOSE"], target_price, stop_loss_price)
          if df.loc[idx1, "CLOSE"] >= target_price:
            count_tp_breach += 1
          elif df.loc[idx1, "CLOSE"] <= stop_loss_price:
            count_sl_breach += 1
          idx1 += 1

        elif df.loc[idx, 'CCI'] >= cci_threshold and df.loc[idx, 'RSI'] >= rsi_threshold:
          # print("F2", df.loc[idx1, "CLOSE"], target_price, stop_loss_price)
          if df.loc[idx1, "CLOSE"] >= target_price:
            count_tp_breach += 1
          elif df.loc[idx1, "CLOSE"] <= stop_loss_price:
            count_sl_breach += 1
          # if df.loc[idx1, 'CCI'] > CCI_max:
            # CCI_max_value = df.loc[idx1, "CLOSE"]
            # CCI_max = df.loc[idx1, 'CCI']
          idx1 += 1

        elif df.loc[idx1, 'CCI'] < cci_threshold or df.loc[idx1, 'RSI'] < rsi_threshold:
          stock_price_indicator_less_threshold = df.loc[idx1, "CLOSE"]
          break
        else:
          idx1 += 1
      
      if stock_price_indicator_less_threshold is not None and exited_value is not None:
        profit_or_loss = stock_price_indicator_less_threshold - exited_value
      else:
        # print("SP", stock_price_indicator_less_threshold, "EV", exited_value)
        profit_or_loss = 0

      if stock in top_25_stocks:
        rows.append([date, stock, buy_price, target_price, stop_loss_price, exited_value, exited_value_date, count_tp_breach, count_sl_breach])

      idx += 1

    else:
      idx += 1


In [10]:
top_25_stocks = set()
top_25_stocks.add('RELIANCE')
top_25_stocks.add('ICICIBANK')
top_25_stocks.add('TATAMOTORS')
top_25_stocks.add('ZOMATO')
top_25_stocks.add('AXISBANK')
top_25_stocks.add('INFY')
top_25_stocks.add('HDFCBANK')
top_25_stocks.add('ITC')
top_25_stocks.add('PAYTM')
top_25_stocks.add('TCS')
top_25_stocks.add('BHARTIARTL')
top_25_stocks.add('INDUSINDBK')
top_25_stocks.add('IRCTC')
top_25_stocks.add('BAJFINANCE')
top_25_stocks.add('SUNPHARMA')
top_25_stocks.add('LT')
top_25_stocks.add('HCLTECH')
top_25_stocks.add('HINDALCO')
top_25_stocks.add('MARUTI')
top_25_stocks.add('TATASTEEL')
top_25_stocks.add('BANDHANBNK')
top_25_stocks.add('HINDUNILVR')
top_25_stocks.add('BPCL')
top_25_stocks.add('WIPRO')
top_25_stocks.add('ONGC')
print(top_25_stocks)

{'ICICIBANK', 'AXISBANK', 'TATASTEEL', 'SUNPHARMA', 'BANDHANBNK', 'ONGC', 'HINDUNILVR', 'HINDALCO', 'ITC', 'RELIANCE', 'ZOMATO', 'IRCTC', 'BAJFINANCE', 'BPCL', 'HDFCBANK', 'TATAMOTORS', 'INFY', 'WIPRO', 'MARUTI', 'TCS', 'LT', 'PAYTM', 'BHARTIARTL', 'INDUSINDBK', 'HCLTECH'}


In [11]:
count = 0

# new_df = pd.read_csv('/content/drive/MyDrive/557 Project - Group 44/Top 25 stocks.xlsx')
# top_25_stocks = new_df['Symbol']

folder_path = '/content/drive/MyDrive/557 Project - Group 44/CCIDataForAllSymbols/'
save_file_name = folder_path + "Tp_Sp_Profit_or_Loss_CCI_new.csv"

headers = ['TIMESTAMP', 'Symbol', 'Buy Price', 'Target Price', 'Stop Loss Price', 'Exited At Price', 'Exited Date', 'Times stock price went above Target Price', 'Times stock price went below Stop Loss Price']
with open(save_file_name, mode='w', newline='') as csv_file:
  writer = csv.writer(csv_file)
  writer.writerow(headers)
  
  rows = []
  for stock in unique_names:
    df = df_all[df_all["SYMBOL"] == stock]
    # print("COUNT=", len(df) ,count)
    indicator_analysis(top_25_stocks, stock, df, count, rows, cci_threshold=110, rsi_threshold=61, percent_profit=30, percent_SL=7)
    count = count + len(df)

  # print(len(rows))  
  for row in rows:
      writer.writerow(row)

Performing For ZOMATO
Performing For WIPRO
Performing For MCDOWELL-N
Performing For ULTRACEMCO
Performing For TORNTPHARM
Performing For TITAN
Performing For TECHM
Performing For TATASTEEL
Performing For TATAPOWER
Performing For TATAMOTORS
Performing For TATACONSUM
Performing For TCS
Performing For SUNPHARMA
Performing For SIEMENS
Performing For SHREECEM
Performing For MOTHERSON
Performing For SRF
Performing For SBILIFE
Performing For SBICARD
Performing For RELIANCE
Performing For PGHH
Performing For POWERGRID
Performing For PIDILITIND
Performing For PIIND
Performing For PAYTM
Performing For ONGC
Performing For NESTLEIND
Performing For MPHASIS
Performing For MARUTI
Performing For MARICO
Performing For M&M
Performing For LICI
Performing For LT
Performing For LTIM
Performing For INDIGO
Performing For INFY
Performing For NAUKRI
Performing For INDUSINDBK
Performing For INDUSTOWER
Performing For IRCTC
Performing For IOC
Performing For ITC
Performing For ICICIPRULI
Performing For ICICIGI
Perf